In [35]:
#import datetime
import glob
import os
from datetime import datetime, timedelta
from pathlib import Path

munge_dir = '/data/scripts/l2munger-main'
py3_path = '/home/tjt/anaconda3/bin/python'
base_dir = '/home/tjt/public_html/public/radar/'
orig_dir = f'{base_dir}/orig'
stage_dir = f'{base_dir}/stage'


class Munger():
    """

    """
    
    def __init__(self,orig_dir,reference_time='19920127_133000',target_rda='KGRR'):
        
        self.orig_dir = Path(orig_dir)
        self.orig_files = list(self.orig_dir.glob('*V06'))
        self.first_file = self.orig_files[0].parts[-1]
        first_file_rda = first_file[:4]
        first_file_time = datetime.strptime(first_file[4:-4], '%Y%m%d_%H%M%S')

        if target_rda is None:
            self.target_rda = first_file_rda
        else:
            self.target_rda = target_rda
        
        self.uncompress_files()
        self.munge_files()
    
    def uncompress_files(self):
        #python debz.py KBRO20170825_195747_V06 KBRO20170825_195747_V06.uncompressed
        for f in self.orig.files:
            command_string = f'{py3_path} debz.py {str(f)} {str(f)}.uncompressed'
        self.uncompressed_files = list(self.orig_files.glob('*uncompressed'))
        return

    def reference_time_shift(self):
        first_file_time_epoch = datetime.strptime(self.first_file[4:-4], '%Y%m%d_%H%M%S').timestamp()
        reference_time_epoch = datetime.strptime(self.reference_time, '%Y%m%d_%H%M%S').timestamp()
        self.time_shift = first_file_time_epoch - reference_time
        return
    
    def munge_files(self):
        """
        Sets times of files to reference time
        """
        for f in self.uncompressed_files:
            fn = str(f.parts[-1])
            file_time = datetime.strptime(fn[4:-4], '%Y%m%d_%H%M%S')
            if self.reference_time is None:
                new_time_obj = file_time - timedelta(seconds=180)
            else:
                self.reference_time_shift();
                new_time_obj = file_time - timedelta(seconds=self.time_shift)
            
            new_time_str = datetime.strftime(new_time_obj, '%Y/%m/%d %H:%M:%S')
            command_line = f'./l2munger {self.new_rda} 1 {new_time_str} {fn}.uncompressed'
        return
    

#-------------------------------

#test = Munger('C:/data/events/wdtd_shv')
test = Munger('/home/tjt/public_html/public/stage', reference_time=None)